<a href="https://colab.research.google.com/github/keerthireddy-28/ExplainableAi/blob/main/LabAssignment_exai_ASS7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
import xgboost as xgb

In [2]:

data = pd.read_csv('/content/loan_data.csv')
data.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [4]:

data.columns

Index(['person_age', 'person_gender', 'person_education', 'person_income',
       'person_emp_exp', 'person_home_ownership', 'loan_amnt', 'loan_intent',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
       'credit_score', 'previous_loan_defaults_on_file', 'loan_status'],
      dtype='object')

In [5]:
data[data.select_dtypes(include='object').columns] = data.select_dtypes(include='object').apply(lambda x: x.str.strip())

In [10]:
data['loan_status'].unique()

array([1, 0])

In [12]:
print("Shape:", data.shape)

Shape: (45000, 14)


In [13]:
# Inspect datatypes
print("\nData types:\n", data.dtypes)


Data types:
 person_age                        float64
person_gender                      object
person_education                   object
person_income                     float64
person_emp_exp                      int64
person_home_ownership              object
loan_amnt                         float64
loan_intent                        object
loan_int_rate                     float64
loan_percent_income               float64
cb_person_cred_hist_length        float64
credit_score                        int64
previous_loan_defaults_on_file     object
loan_status                         int64
dtype: object


In [14]:
# Check missing values
print("\nMissing values:\n", data.isnull().sum())


Missing values:
 person_age                        0
person_gender                     0
person_education                  0
person_income                     0
person_emp_exp                    0
person_home_ownership             0
loan_amnt                         0
loan_intent                       0
loan_int_rate                     0
loan_percent_income               0
cb_person_cred_hist_length        0
credit_score                      0
previous_loan_defaults_on_file    0
loan_status                       0
dtype: int64


In [16]:
print(data['loan_status'].value_counts())

loan_status
0    35000
1    10000
Name: count, dtype: int64


In [17]:
for col in data.select_dtypes(include="number").columns:
    data[col].fillna(data[col].median(), inplace=True)
for col in data.select_dtypes(include="object").columns:
    data[col].fillna(data[col].mode()[0], inplace=True)

/tmp/ipython-input-531700803.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
/tmp/ipython-input-531700803.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [18]:

# Encode categorical columns
le = LabelEncoder()
for col in data.select_dtypes(include="object").columns:
    data[col] = le.fit_transform(data[col])

In [21]:
# Features and target
X = data.drop('loan_status', axis=1)
y = data['loan_status']

In [22]:
# Scale numeric features
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [23]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [25]:
# Handle imbalance with SMOTE
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [26]:
print(y_train.value_counts())       # before SMOTE
print(y_train_res.value_counts())   # after SMOTE

loan_status
0    28000
1     8000
Name: count, dtype: int64
loan_status
0    28000
1    28000
Name: count, dtype: int64


In [27]:
# Logistic Regression
lr = LogisticRegression(class_weight="balanced", random_state=42)
lr.fit(X_train_res, y_train_res)

# Random Forest
rf = RandomForestClassifier(class_weight="balanced", random_state=42)
rf.fit(X_train_res, y_train_res)

# XGBoost
xgb_model = xgb.XGBClassifier(scale_pos_weight=(y_train_res.value_counts()[0]/y_train_res.value_counts()[1]), random_state=42, use_label_encoder=False, eval_metric="logloss")
xgb_model.fit(X_train_res, y_train_res)


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [04:32:09] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [28]:
#Evaluate models
models = {"Logistic Regression": lr, "Random Forest": rf, "XGBoost": xgb_model}

for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"{name}")
    print(classification_report(y_test, y_pred))
    print("\n")

Logistic Regression
              precision    recall  f1-score   support

           0       0.97      0.83      0.90      7000
           1       0.61      0.91      0.73      2000

    accuracy                           0.85      9000
   macro avg       0.79      0.87      0.82      9000
weighted avg       0.89      0.85      0.86      9000



Random Forest
              precision    recall  f1-score   support

           0       0.95      0.94      0.95      7000
           1       0.80      0.84      0.82      2000

    accuracy                           0.92      9000
   macro avg       0.88      0.89      0.88      9000
weighted avg       0.92      0.92      0.92      9000



XGBoost
              precision    recall  f1-score   support

           0       0.95      0.96      0.96      7000
           1       0.86      0.83      0.85      2000

    accuracy                           0.93      9000
   macro avg       0.90      0.90      0.90      9000
weighted avg       0.93     

In [29]:
!pip install dice-ml --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.5 MB/s eta 0:00:00


In [30]:
import dice_ml
from dice_ml.utils import helpers


In [32]:
# 1. Clean column names
data.columns = data.columns.str.strip()

# 2. Encode categorical columns only
le = LabelEncoder()
for col in ['person_gender', 'person_education', 'person_home_ownership', 'loan_intent', 'previous_loan_defaults_on_file']:
    data[col] = le.fit_transform(data[col])

# 3. Split features and target (unscaled)
X = data.drop("loan_status", axis=1)
y = data["loan_status"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 4. SMOTE (optional)
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# 5. Train Random Forest on **same features used in DiCE**
rf = RandomForestClassifier(class_weight="balanced", random_state=42)
rf.fit(X_train_res, y_train_res)

# 6. Now DiCE query instance from X_test will match feature names exactly
query_instance = X_test[y_test==0].iloc[[0]]  # first rejected

In [34]:
# Imports
import dice_ml
import pandas as pd

# 1. Fix column spaces
data.columns = data.columns.str.strip()

# 2. Define continuous features (corrected)
continuous_features = [
    'person_age', 'person_income', 'person_emp_exp', 'loan_amnt',
    'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
    'credit_score'
]

# 3. Create DiCE Data object (original, unscaled)
d = dice_ml.Data(
    dataframe=data,
    continuous_features=continuous_features,
    outcome_name='loan_status'
)

# 4. Create DiCE Model object using trained Random Forest
m = dice_ml.Model(model=rf, backend='sklearn')

# 5. Initialize DiCE explainer
exp = dice_ml.Dice(d, m)

# 6. Pick a test instance predicted as negative (Rejected)
#    Use original unscaled data and drop target column
neg_idx = y_test[y_test==0].index[0]  # first Rejected
query_instance = data.loc[[neg_idx]].drop(columns=['loan_status'])  # only features

# 7. Generate 3 counterfactuals
cf_examples = exp.generate_counterfactuals(
    query_instance,
    total_CFs=3,
    desired_class="opposite"
)

# 8. Visualize counterfactuals, showing only changed features
cf_examples.visualize_as_dataframe(show_only_changes=True)
cf_df = cf_examples.cf_examples_list[0].final_cfs_df
print("Counterfactuals vs Original:\n", cf_df)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
/usr/local/lib/python3.12/dist-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
/usr/local/lib/python3.12/dist-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is depr

Query instance (original outcome : 0)


,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,25.0,0,1,84973.0,2,0,14000.0,5,5.42,0.16,3.0,634,0,0



Diverse Counterfactual set (new outcome: 1)


,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,-,-,-,-,-,-,-,-,17.99,-,27.9,-,-,1.0
1,-,-,-,-,-,3.0,-,-,16.5,-,-,-,-,1.0
2,99.1,-,-,-,-,-,-,-,15.26,-,-,-,-,1.0


Counterfactuals vs Original:
    person_age  person_gender  person_education  person_income  person_emp_exp  \
0        25.0              0                 1        84973.0               2   
1        25.0              0                 1        84973.0               2   
2        99.1              0                 1        84973.0               2   

   person_home_ownership  loan_amnt  loan_intent  loan_int_rate  \
0                      0    14000.0            5          17.99   
1                      3    14000.0            5          16.50   
2                      0    14000.0            5          15.26   

   loan_percent_income  cb_person_cred_hist_length  credit_score  \
0                 0.16                        27.9           634   
1                 0.16                         3.0           634   
2                 0.16                         3.0           634   

   previous_loan_defaults_on_file  loan_status  
0                               0            1  
1    

In [35]:
query_instance_reindexed = query_instance.iloc[0].reindex(cf_df.columns)
changed_features = cf_df.loc[:, (cf_df != query_instance_reindexed).any()]
influential_features = changed_features.columns.tolist()
print("Most influential features:", influential_features)

Most influential features: ['person_age', 'person_home_ownership', 'loan_int_rate', 'cb_person_cred_hist_length', 'loan_status']


In [36]:
# 9. Generate counterfactuals using Manhattan distance
cf_examples_manhattan = exp.generate_counterfactuals(
    query_instance, total_CFs=3, desired_class="opposite"
)
cf_examples_manhattan.visualize_as_dataframe(show_only_changes=True)
cf_df_manhattan = cf_examples_manhattan.cf_examples_list[0].final_cfs_df
print("Counterfactuals with Manhattan distance:\n", cf_df_manhattan)

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
/usr/local/lib/python3.12/dist-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
/usr/local/lib/python3.12/dist-packages/dice_ml/explainer_interfaces/dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is depr

Query instance (original outcome : 0)


,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,25.0,0,1,84973.0,2,0,14000.0,5,5.42,0.16,3.0,634,0,0



Diverse Counterfactual set (new outcome: 1)


,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,-,-,-,-,-,-,-,-,15.54,-,-,-,-,-
1,-,-,-,-,-,-,-,0.0,12.38,-,-,-,-,1.0
2,-,-,-,-,-,-,-,1.0,16.62,-,-,-,-,1.0


Counterfactuals with Manhattan distance:
    person_age  person_gender  person_education  person_income  person_emp_exp  \
0        25.0              0                 1        84973.0               2   
1        25.0              0                 1        84973.0               2   
2        25.0              0                 1        84973.0               2   

   person_home_ownership  loan_amnt  loan_intent  loan_int_rate  \
0                      0    14000.0            5          15.54   
1                      0    14000.0            0          12.38   
2                      0    14000.0            1          16.62   

   loan_percent_income  cb_person_cred_hist_length  credit_score  \
0                 0.16                         3.0           634   
1                 0.16                         3.0           634   
2                 0.16                         3.0           634   

   previous_loan_defaults_on_file  loan_status  
0                               0         